In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
import re

In [2]:
# read data
train_data = pd.read_csv("Train.csv", sep=',')
test_data = pd.read_csv("Test.csv", sep=',')

In [3]:
#separating instance and label for Train
X_train_raw = [x[0] for x in train_data[['text']].values]
Y_train = [x[0] for x in train_data[['sentiment']].values]
X_test_raw = [x[0] for x in test_data[['text']].values]

In [14]:
# 1. data cleaning (optional)
X_train_need_to_clean = pd.DataFrame(X_train_raw)
X_test_need_to_clean = pd.DataFrame(X_test_raw)

# remove url, # and @
X_train_need_to_clean.replace("\b*https?:\S*", '', regex=True, inplace=True)
X_train_need_to_clean.replace("\b*@\S*", '', regex=True, inplace=True)
X_train_need_to_clean.replace("\b*#\S*", '', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*https?:\S*", '', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*@\S*", '', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*#\S*", '', regex=True, inplace=True)


X_train_clean = [x[0] for x in X_train_need_to_clean[[0]].values]
X_test_clean = [x[0] for x in X_test_need_to_clean[[0]].values]

In [15]:
# 2. vectorization (transformation)

# bag of words
# countvectorizer
BoW_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2))
X_train_BoW = BoW_vectorizer.fit_transform(X_train_clean)
X_test_BoW = BoW_vectorizer.transform(X_test_clean)

In [16]:
# 3. feature selection
X_train_new = SelectKBest(chi2,k=5000).fit_transform(X_train_BoW,Y_train)


In [17]:
# 4. split
train_size = X_train_BoW.shape[0]
test_size = X_test_BoW.shape[0]
## random hold out
ts = test_size/train_size
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train_new,Y_train, test_size=ts)

# modelling

### base model: 0R

In [18]:
clf = DummyClassifier(strategy='most_frequent')
basemodel = clf.fit(X_train_raw, Y_train)
print("base model score: ", basemodel.score(X_train_raw, Y_train))

base model score:  0.5806348041464086


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

### logistic regression

###### choose of hyperparameter

In [21]:

hyper = {
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'penalty': ['l1', 'l2', 'none', 'elasticnet'],
    'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
    'max_iter': [100, 200, 300],
    'multi_class': ['auto', 'ovr', 'multinomial']
}

# clf = GridSearchCV(LogisticRegression(),hyper, scoring='accuracy', cv=5)
search_logi = RandomizedSearchCV(LogisticRegression(),hyper, scoring='accuracy', cv=5, n_iter=10)
result = search_logi.fit(X_train_s, y_train_s)



D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\utils\optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warn

D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
D:\Users\laizh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

In [24]:
pd.DataFrame(result.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_penalty,param_multi_class,param_max_iter,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.137532,4.394632e-02,0.002000,1.168008e-07,saga,l2,multinomial,200,0.01,"{'solver': 'saga', 'penalty': 'l2', 'multi_cla...",0.597899,0.598535,0.593442,0.592357,0.596497,0.595746,0.002440,7
1,0.657758,2.929623e-02,0.001800,3.999949e-04,lbfgs,none,ovr,200,10,"{'solver': 'lbfgs', 'penalty': 'none', 'multi_...",0.712512,0.715059,0.715696,0.715924,0.710828,0.714004,0.001999,1
2,0.381800,2.837976e-03,0.002001,3.371748e-07,saga,none,ovr,100,1,"{'solver': 'saga', 'penalty': 'none', 'multi_c...",0.679083,0.681312,0.678128,0.690127,0.701592,0.686049,0.008854,4
3,0.001400,4.900961e-04,0.000000,0.000000e+00,sag,elasticnet,auto,200,10,"{'solver': 'sag', 'penalty': 'elasticnet', 'mu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
4,18.601260,1.402999e+00,0.002201,3.999949e-04,saga,l1,multinomial,300,1,"{'solver': 'saga', 'penalty': 'l1', 'multi_cla...",0.637695,0.635148,0.629099,0.643949,0.639490,0.637076,0.004915,6
5,35.909399,5.768511e+00,0.002400,4.900961e-04,newton-cg,none,multinomial,200,10,"{'solver': 'newton-cg', 'penalty': 'none', 'mu...",0.695320,0.698185,0.702324,0.699045,0.691720,0.697319,0.003581,3
6,0.001600,4.902907e-04,0.000000,0.000000e+00,lbfgs,l1,ovr,200,0.001,"{'solver': 'lbfgs', 'penalty': 'l1', 'multi_cl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,1.195603,4.679094e-02,0.002200,4.000664e-04,lbfgs,none,multinomial,300,0.01,"{'solver': 'lbfgs', 'penalty': 'none', 'multi_...",0.703598,0.708691,0.710602,0.713057,0.705096,0.708209,0.003476,2
8,0.367383,3.036747e-02,0.002601,4.899598e-04,newton-cg,l2,multinomial,200,1,"{'solver': 'newton-cg', 'penalty': 'l2', 'mult...",0.676855,0.674626,0.674944,0.684395,0.684076,0.678979,0.004360,5
9,0.002000,1.168008e-07,0.000000,0.000000e+00,newton-cg,l1,ovr,300,0.1,"{'solver': 'newton-cg', 'penalty': 'l1', 'mult...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10


In [23]:
result.best_params_

{'solver': 'lbfgs',
 'penalty': 'none',
 'multi_class': 'ovr',
 'max_iter': 200,
 'C': 10}